# Modeling Historical Temperature Using Milankovitch Cycles
This notebook uses Bayesian methods and MCMC (via `emcee`) to fit a sinusoidal model to paleoclimate data extracted from ice core measurements. The model includes three periodic components reflecting Milankovitch cycles.

## ⚠️ Data Notice
*The original dataset `ice_core_data.txt` used in this project is no longer available. However, the code below remains functional and ready to use with any compatible dataset.*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import pymc as pm
import os
from matplotlib.ticker import ScalarFormatter

plt.rcParams['figure.figsize'] = (20,10)

## Load the Dataset
This code assumes that the file `ice_core_data.txt` is in the same directory. The dataset should contain at least the age (years before present) and temperature deviation columns.

In [ ]:
# ⚠️ This code is commented out since the dataset is not available.
# Uncomment and run if you have the dataset.
# current_dir = os.path.dirname(os.path.abspath("ice_core_data.txt"))
# file_path = os.path.join(current_dir, 'ice_core_data.txt')
# ice_data = np.loadtxt(file_path)
# ice_data = np.transpose(ice_data)
# age = ice_data[2]
# T = ice_data[4]

## Define the Model
The model consists of three sinusoidal components and a constant offset.

In [ ]:
def model(theta, age):
    a1, a2, a3, p1, p2, p3, T0 = theta
    return (a1 * np.sin(2 * np.pi * age / p1) +
            a2 * np.sin(2 * np.pi * age / p2) +
            a3 * np.sin(2 * np.pi * age / p3) + T0)

In [ ]:
def lnlike(theta, x, y, yerr):
    return -0.5 * np.sum(((y - model(theta, x)) / yerr) ** 2)

In [ ]:
def lnprior(theta):
    a1, a2, a3, p1, p2, p3, T0 = theta
    if 0.0 < a1 < 5.0 and 0.0 < a2 < 5.0 and 0.0 < a3 < 5.0 and \
       10000. < p1 < 200000 and 10000. < p2 < 200000 and 10000. < p3 < 200000 and \
       -10.0 < T0 < 0:
        return 0.0
    return -np.inf

In [ ]:
def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)

## MCMC Sampling Setup
Set the number of walkers, iterations, and initial parameters.

In [ ]:
Terr = 0.05  # Placeholder for mean temperature error
# data = (age, T, Terr)
nwalkers = 128
niter = 500
initial = np.array([1.0, 1.0, 1.0, 26000., 41000., 100000., -4.5])
ndim = len(initial)
p0 = [initial + 1e-7 * np.random.randn(ndim) for _ in range(nwalkers)]

In [ ]:
def run_mcmc(p0, nwalkers, niter, ndim, lnprob, data):
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=data)
    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(p0, 100)
    sampler.reset()
    print("Running production...")
    pos, prob, state = sampler.run_mcmc(p0, niter)
    return sampler, pos, prob, state

## Visualization
Plot the model fit with randomly selected samples.

In [ ]:
def plot_samples(sampler, age, T):
    plt.plot(age, T, label='Observed')
    samples = sampler.flatchain
    for theta in samples[np.random.randint(len(samples), size=100)]:
        plt.plot(age, model(theta, age), color='r', alpha=0.1)
    plt.xlabel('Years ago')
    plt.ylabel('ΔT (°C)')
    plt.legend()
    plt.show()

## Summary
This notebook demonstrates how to apply Bayesian inference via MCMC to fit a composite sinusoidal model to paleoclimate data. Despite the dataset not being available, the code can be reused and adapted to other datasets of similar structure.